## Shift + tab for methods info
## command + / = comments out all

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

import datetime as dt
import numpy as np
import os

# Load packages
from pygeocoder import Geocoder


In [3]:
os.listdir()

['MyETLProject_2010-Present.ipynb',
 'ETL_project.sql',
 'CrimeData_2010-Present.csv',
 'MyETLProject_2010-Present_05.17.19.ipynb',
 'MyETLProject_2010-Present-Copy1.ipynb',
 'MyETLProject_2010-Present_05.18.19.ipynb',
 '.ipynb_checkpoints',
 'diffs_latlong.py',
 'us-zip-code-latitude-and-longitude.json']

In [4]:
#read csv file
df = pd.read_csv("CrimeData_2010-Present.csv",parse_dates=True)

In [5]:
df.head()
df.tail()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
1048570,171518209,9/3/17,9/2/17,1500,15,N Hollywood,1501,510,VEHICLE - STOLEN,NaN,...,NaN,AO,Adult Other,510.0,NaN,NaN,NaN,SATICOY,VARNA,"(34.2087, -118.4247)"
1048571,171518210,9/3/17,9/3/17,1500,15,N Hollywood,1501,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),NaN,...,NaN,IC,Invest Cont,420.0,NaN,NaN,NaN,ARMINTA ST,AMPERE AV,"(34.213, -118.4189)"
1048572,171518213,9/3/17,9/3/17,1910,15,N Hollywood,1504,624,BATTERY - SIMPLE ASSAULT,0913 0400,...,UNKNOWN WEAPON/OTHER WEAPON,AA,Adult Arrest,624.0,NaN,NaN,NaN,11900 RUNNYMEDE ST,NaN,"(34.2058, -118.3921)"
1048573,171518214,9/3/17,9/3/17,1300,15,N Hollywood,1586,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1259,...,NaN,JA,Juv Arrest,745.0,NaN,NaN,NaN,10600 VALLEYHEART DR,NaN,"(34.1419, -118.3609)"
1048574,171518215,9/3/17,9/3/17,1730,15,N Hollywood,1591,626,INTIMATE PARTNER - SIMPLE ASSAULT,2000 1814 0416,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AA,Adult Arrest,626.0,NaN,NaN,NaN,12600 VENTURA BL,NaN,"(34.1437, -118.4074)"


In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.shape

(1048575, 26)

## Check variables

In [8]:
#print the dataframe columns

cols= list(df.columns)
print(cols)

['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area ID', 'Area Name', 'Reporting District', 'Crime Code', 'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Code', 'Premise Description', 'Weapon Used Code', 'Weapon Description', 'Status Code', 'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4', 'Address', 'Cross Street', 'Location ']


In [12]:
#remove spaces from the variable name by assigning the values to a new variable
df['Location'] = df['Location ']


In [25]:
#removed old column
df.drop(columns=['Location '], axis=1, inplace = True)


In [26]:
#check column changes
df.columns

Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code', 'Premise Description',
       'Weapon Used Code', 'Weapon Description', 'Status Code',
       'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3',
       'Crime Code 4', 'Address', 'Cross Street', 'Location'],
      dtype='object')

In [80]:
df['Crime Code Description'].head()

0    VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...
1                            OTHER MISCELLANEOUS CRIME
2                                BURGLARY FROM VEHICLE
3    THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...
4    VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...
Name: Crime Code Description, dtype: object

In [81]:
df['Address'].head()

0            SEPULVEDA                    BL
1    1300 E  21ST                         ST
2            PICO                         BL
3     200 S  OLIVE                        ST
4    1100 S  OLIVE                        ST
Name: Address, dtype: object

In [29]:
#check content of the Location variable
     #has latitud and longitude together
df['Location'].head()

0    (33.9599, -118.3962)
1    (34.0224, -118.2524)
2    (34.0389, -118.2643)
3    (34.0538, -118.2488)
4    (34.0409, -118.2609)
Name: Location, dtype: object

In [30]:
# Separate the content of the Location variable. Latitude and longitude should be separate variables.

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in df['Location']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
df['latitude']= lat
df['longitude'] = lon

In [31]:
#check results

df['latitude'].head()

#parenthesis should be removed from the variable

0    (33.9599
1    (34.0224
2    (34.0389
3    (34.0538
4    (34.0409
Name: latitude, dtype: object

In [32]:
#check results

df['longitude'].head()

#parenthesis should be removed from the variable

0     -118.3962)
1     -118.2524)
2     -118.2643)
3     -118.2488)
4     -118.2609)
Name: longitude, dtype: object

In [33]:
#remove parenthesis of the latitude variable with the replace method

df['latitude'].replace(regex=True,inplace=True,to_replace=r'\(',value=r'')
df['latitude'].head()

0    33.9599
1    34.0224
2    34.0389
3    34.0538
4    34.0409
Name: latitude, dtype: object

In [34]:
#remove parenthesis of the longitude variable with the replace method

df['longitude'].replace(regex=True,inplace=True,to_replace=r'\)',value=r'')
df['longitude'].head()

0     -118.3962
1     -118.2524
2     -118.2643
3     -118.2488
4     -118.2609
Name: longitude, dtype: object

In [35]:
#check results
df[['latitude', 'longitude']].head()

,latitude,longitude
0,33.9599,-118.3962
1,34.0224,-118.2524
2,34.0389,-118.2643
3,34.0538,-118.2488
4,34.0409,-118.2609


In [36]:
#check data type 
df['latitude'].dtype

dtype('O')

In [37]:
#change data type of the latitude and longitude column

df['latitude']=df['latitude'].astype(str).astype(float)
df['longitude']= df['longitude'].astype(str).astype(float)

In [38]:
#check change
df['latitude'].dtype

dtype('float64')

In [39]:
#check change
df['longitude'].dtype

dtype('float64')

In [40]:
df.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                  int64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Weapon Used Code          float64
Weapon Description         object
Status Code                object
Status Description         object
Crime Code 1              float64
Crime Code 2              float64
Crime Code 3              float64
Crime Code 4              float64
Address                    object
Cross Street               object
Location                   object
latitude                  float64
longitude                 float64
dtype: object

In [41]:
#test the function to get zip code from latitude and longitude

from math import sin, cos, sqrt, atan2, radians
import math
import json
fname = 'us-zip-code-latitude-and-longitude.json'
loadedLatLongData = json.load(open(fname))


def distanceV(lat1, lon1,lat2, lon2):
    #lat1, lon1 = origin
    #lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

def computeNearestZip(latitude, longitude):
    smallestDistance = 1000000
    smallestZip = None
    for data in loadedLatLongData:
        if ("fields" in data):
            distancedata = data["fields"]
            if("latitude" in distancedata and "longitude" in distancedata and "zip" in distancedata):
                zipcode = distancedata["zip"]
                lat1 = distancedata["latitude"]
                long1 = distancedata["longitude"]
                distance = distanceV(latitude,longitude, lat1,long1)

                if(distance <smallestDistance):
                    smallestZip = zipcode
                    smallestDistance = distance


    return smallestDistance, smallestZip

if __name__ == '__main__':
    #lat long of 30339
    smallestDistance, smallestZip = computeNearestZip(33.9599,-118.3962)
    print(smallestZip)
    print(smallestDistance)

    #lat long of 27510
    smallestDistance, smallestZip = computeNearestZip(35.9108, -79.0815)
    print(smallestZip)
    print(smallestDistance)

90045
0.1209174146010057
27510
0.18788594968216657


In [80]:
 computeNearestZip(33.9599, -118.3962)

(0.1209174146010057, '90045')

In [42]:
# df['latitude'].dtype

dtype('float64')

In [43]:
# df['longitude'].dtype

dtype('float64')

In [183]:
# def get_element(my_list, position):
#     return my_list[position]

In [203]:
df['Location'][0:1].str.split(',')

0    [33.9599,  -118.3962]
Name: Location, dtype: object

In [45]:
df['Location'].head()

0    (33.9599, -118.3962)
1    (34.0224, -118.2524)
2    (34.0389, -118.2643)
3    (34.0538, -118.2488)
4    (34.0409, -118.2609)
Name: Location, dtype: object

In [46]:
# remove left parenthesis from the Location variable
df['Location'].replace(regex=True,inplace=True,to_replace=r'\)',value=r'')

In [48]:
# remove right parenthesis from the Location variable
df['Location'].replace(regex=True,inplace=True,to_replace=r'\(',value=r'')

In [50]:
df['Location'].replace(regex=True,inplace=True,to_replace=r'\,',value=r'')

In [51]:
#check results
df['Location'].head()

0    33.9599 -118.3962
1    34.0224 -118.2524
2    34.0389 -118.2643
3    34.0538 -118.2488
4    34.0409 -118.2609
Name: Location, dtype: object

In [54]:
df['Location'][0:1]

0    33.9599 -118.3962
Name: Location, dtype: object

In [59]:
df['Zipcode'] = df['Location'][0:1].apply([func='computeNearestZip', axis=1, broadcast=True, raw=False])

SyntaxError: invalid syntax (<ipython-input-59-c72301ab45e2>, line 1)

In [71]:
df['Zipcode'] = df['Location'].apply(func=[computeNearestZip],axis=1, raw=False)

ValueError: No axis named 1 for object type <class 'pandas.core.series.Series'>

In [72]:
df['Zipcode'] = df['Location'].apply(func=[computeNearestZip], raw=False, args())

ValueError: no results

In [81]:
df['Zipcode'] = df['Location'].apply(func=[computeNearestZip], raw=False)

ValueError: no results

In [76]:
df[['latitude','longitude']] = df[['latitude','longitude']].apply(func=[computeNearestZip], raw=False, axis=0)

ValueError: no results

In [79]:
df[['latitude','longitude']] = df.apply(func=computeNearestZip, raw=False, axis=1)

TypeError: ("computeNearestZip() missing 1 required positional argument: 'longitude'", 'occurred at index 0')

In [197]:
df[['latitude','longitude']][0:1]

,latitude,longitude
0,33.9599,-118.3962


In [195]:
df[Zipcode] = df[['latitude','longitude']].apply(computeNearestZip, axis=1)

#df['a'] = df['a'].apply(lambda x: x + 1)
#df.apply(computeNearestZip(df["latitude"],df["longitude"]), convert_dtype=True, args=())

TypeError: ("computeNearestZip() missing 1 required positional argument: 'longitude'", 'occurred at index 0')

In [144]:
df[['latitude','longitude']] = df[['latitude','longitude']].apply(lambda[['latitude','longitude']]: computeNearestZip(), convert_dtype=True, args=())


SyntaxError: invalid syntax (<ipython-input-144-520bc3ed5bd8>, line 1)

In [ ]:
https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY

In [ ]:
# webpage resources

#https://stackoverflow.com/questions/23307301/replacing-column-values-in-a-pandas-dataframe
#https://stackoverflow.com/questions/13682044/remove-unwanted-parts-from-strings-in-a-column
#https://stackoverflow.com/questions/3559559/how-to-delete-a-character-from-a-string-using-python
#https://stackoverflow.com/questions/5043815/python-parsing-columns-and-rows